<a href="https://colab.research.google.com/github/infocz-lucy/neodash-fork/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PDF 처리 및 OCR
!pip install pdfplumber pdf2image pytesseract

# DOCX 처리
!pip install python-docx

# 텍스트 전처리
!pip install konlpy nltk spacy transformers

# RDF 변환
!pip install rdflib

# Neo4j Python 드라이버
!pip install neo4j

# OpenAI API
!pip install openai

# 한국어 형태소 분석기 Mecab 설치 (Google Colab 전용)
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# mecab-python의 버전 오류로 인해 아래 패키지를 설치하면 코랩에서 Mecab을 사용가능
!pip install mecab-python3

# SpaCy 영어 모델 다운로드
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 856.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=71db41b

In [12]:
# 필요한 라이브러리 임포트
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
from docx import Document
from konlpy.tag import Mecab
import re
from nltk.corpus import stopwords
import nltk
import spacy

# 한국어 형태소 분석기 및 영어 NLP 모델 로드
mecab = Mecab()
nlp_en = spacy.load("en_core_web_sm")
nltk.download("stopwords")
stopwords_en = set(stopwords.words("english"))

# PDF 추출 및 OCR
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    if not text:  # PDF가 이미지인 경우 OCR 사용
        images = convert_from_path(pdf_path, 500)
        for img in images:
            text += pytesseract.image_to_string(img, lang="kor+eng")
    return text.strip()

# DOCX 추출
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

# 텍스트 전처리 (한국어)
def preprocess_korean(text):
    text = re.sub(r"[^가-힣\s]", "", text)
    tokens = mecab.morphs(text)
    return tokens

# 텍스트 전처리 (영어)
def preprocess_english(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text.lower())
    tokens = [word for word in text.split() if word not in stopwords_en]
    return tokens

# 예시 PDF 및 DOCX 파일 경로
pdf_path = "/content/sample_data/raw.pdf"
docx_path = "example.docx"

# 데이터 추출 및 전처리 실행
text_pdf = extract_text_from_pdf(pdf_path)
# text_docx = extract_text_from_docx(docx_path)

tokens_pdf_ko = preprocess_korean(text_pdf)
tokens_pdf_en = preprocess_english(text_pdf)

# tokens_docx_ko = preprocess_korean(text_docx)
# tokens_docx_en = preprocess_english(text_docx)

print("PDF 한국어 토큰:", tokens_pdf_ko)
print("PDF 영어 토큰:", tokens_pdf_en)
# print("DOCX 한국어 토큰:", tokens_docx_ko)
# print("DOCX 영어 토큰:", tokens_docx_en)


PDF 한국어 토큰: ['제장', '총강', '제조', '대한민국', '은', '민주', '공화국', '이', '다', '대한민국', '의', '주권', '은', '국민', '에게', '있', '고', '모든', '권력', '은', '국민', '으로부터', '나온다', '제조', '대한민국', '의', '국민', '이', '되', '는', '요건', '은', '법률', '로', '정한다', '국가', '는', '법률', '이', '정하', '는', '바', '에', '의하', '여', '재외', '국민', '을', '보호', '할', '의무', '를', '진다', '제조', '대한민국', '의', '영토', '는', '한반도', '와', '그', '부속', '도', '서로', '한다', '제조', '대한민국', '은', '통일', '을', '지향', '하', '며', '자유', '민주', '적', '기본', '질서', '에', '입각', '한', '평화', '적', '통일', '정책', '을', '수립', '하', '고', '이', '를', '추진', '한다', '제조', '대한민국', '은', '국제', '평화', '의', '유지', '에', '노력', '하', '고', '침략', '적', '전쟁', '을', '부인', '한다', '국군', '은', '국가', '의', '안전', '보장', '과', '국토방위', '의', '신성', '한', '의무', '를', '수행', '함', '을', '사명', '으로', '하', '며', '그', '정치', '적', '중', '립', '성', '은', '준수', '된다', '제조', '헌법', '에', '의하', '여', '체결', '공포', '된', '조약', '과', '일반', '적', '으로', '승인', '된', '국제', '법규', '는', '국내법', '과', '같', '은', '효력', '을', '가', '진다', '외국인', '은', '국제법', '과', '조약', '이', '정하', '는', '바', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
